

Import libraries



In [ ]:
import h5py
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 
import time
from scipy.optimize import minimize,fmin_tnc,fmin,fmin_l_bfgs_b,fmin_cobyla,fmin_slsqp


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/
!pwd

with h5py.File('images_training.h5','r') as H:
    data_train = np.copy(H['datatrain'])
with h5py.File('labels_training.h5','r') as H:
    label_train = np.copy(H['labeltrain'])
with h5py.File('images_testing.h5','r') as H:
    data_test = np.copy(H['datatest'])
with h5py.File('labels_testing_2000.h5','r') as H:
    label_test = np.copy(H['labeltest'])
print(data_train.shape)
print(label_train.shape)
print(data_test.shape)
print(label_test.shape)

Mounted at /content/gdrive
/content/gdrive/My Drive
/content/gdrive/My Drive
(30000, 784)
(30000,)
(5000, 784)
(2000,)


Loading data

In [ ]:
print(os.listdir("./Input/test"))
print(os.listdir("./Input/train"))
#Loading training data
with h5py.File('./Input/train/images_training.h5','r') as H:
    data_train = np.copy(H['datatrain'])
with h5py.File('./Input/train/labels_training.h5','r') as H:
    label_train = np.copy(H['labeltrain'])
print(data_train.shape)
print(label_train.shape)

#Loading testing data
with h5py.File('./Input/test/images_testing.h5','r') as H:
    data_test = np.copy(H['datatest'])
with h5py.File('./Input/test/labels_testing_2000.h5','r') as H:
    label_test = np.copy(H['labeltest'])
print(data_test.shape)
print(label_test.shape)



['labels_testing_2000.h5', 'images_testing.h5']
['images_training.h5', 'labels_training.h5']
(30000, 784)
(30000,)
(5000, 784)
(2000,)


Assign to variables

In [ ]:
training_data = data_train
testing_data = data_test
training_label = label_train
testing_label = label_test

print(training_data.shape)
print(training_label.shape)
print(testing_data.shape)
print(testing_label.shape)

(30000, 784)
(30000,)
(5000, 784)
(2000,)


Preprocessing 

In [ ]:
# Function to scale training data 
def standardize(array):
  x = array 
  x-=np.mean(x)
  x/=np.std(x)
  standardize_array = x
  return standardize_array
# Function to scale training data 
def standardize_test(array,training_data):
  x = array 
  x-=np.mean(training_data)
  x/=np.std(training_data)
  standardize_array = x
  return standardize_array

# Function to reduce the dimensions of data using PCA 
def pca_train(data):
  # Covariance matrix
  covariance_matrix = np.cov(data.T)
  # print(covariance_matrix)

  # Eigen Value decomposition 
  eigen_values, eigen_vectors = np.linalg.eig(covariance_matrix)
  # print("Eigenvector: \n",eigen_vectors,"\n")
  # print("Eigenvalues: \n", eigen_values, "\n")
  # print(eigen_values.shape)
  # print(eigen_vectors.shape) #principal components 

  # Variance calculation 
  variance_explained = []
  for i in eigen_values:
      variance_explained.append((i/sum(eigen_values))*100)
  cumulative_variance_explained = np.cumsum(variance_explained)
  np.around(cumulative_variance_explained,2,out=cumulative_variance_explained)
  # plt.plot(range(1,eigen_values.shape[0]+1),cumulative_variance_explained)
  # plt.xlabel("Number of components")
  # plt.ylabel("Cumulative explained variance")
  # plt.title("Explained variance vs Number of components")
  
  # Feature selection 
  no_of_features = np.where(cumulative_variance_explained > 95)[0][0]
  # print(no_of_features)

  # Projection matrix using the features
  projection_matrix = (eigen_vectors.T[:][:no_of_features]).T
  # print(projection_matrix.shape)

  # Data with reduced dimensions
  data_pca = data.dot(projection_matrix)
  # print(data_train_std_pca.shape)

  return data_pca,projection_matrix

def pca_test(data,projection_matrix):
  # Data with reduced dimensions
  data_pca = data.dot(projection_matrix)
  # print(data_train_std_pca.shape)
  return data_pca

# Function to preprocess training data
def preprocessing_training_data(data_train):
  #standardizing 
  data_train_std = standardize(data_train)
  #PCA
  data_train_std_pca,projection_matrix = pca_train(data_train_std)

  return data_train_std_pca,projection_matrix

# Function to preprocess testing data
def preprocessing_testing_data(data_test,data_train,projection_matrix):
  #standardizing 
  data_test_std = standardize_test(data_test,data_train)
  #PCA
  data_test_std_pca = pca_test(data_test_std,projection_matrix)
  return data_test_std_pca


data_train_std_pca,projection_matrix = preprocessing_training_data(training_data)
print(data_train_std_pca.shape)

data_test_std_pca = preprocessing_testing_data(testing_data,training_data,projection_matrix)
print(data_test_std_pca.shape)

(30000, 187)
(5000, 187)


In [ ]:
training_data = data_train_std_pca
testing_data = data_test_std_pca
training_label = label_train
testing_label = label_test

# training_data = data_train_std_pca[:21000]
# validation_data = data_test_std_pca[21000:]
# training_label = label_train[:21000]
# validation_label = label_test[21000:]
print(training_data.shape)
print(training_label.shape)
print(testing_data.shape)
print(testing_label.shape)

(30000, 187)
(30000,)
(5000, 187)
(2000,)


KNN Classifier 

In [ ]:
class knnclassifier:
  def __init__(self,train_data,train_label):
    self.train_data = train_data 
    self.train_label = train_label

  # Function to calculate the k nearest neighbours and return the mode class from the k nearest neighbours
  def knn(self,x_instance,k):
    dis = ((self.train_data - x_instance)**2).sum(axis=1)
    arg_ascending = np.argsort(dis)
    k_neighbours = arg_ascending[:k]
    k_neighbours_class = self.train_label[k_neighbours]
    u, c = np.unique(k_neighbours_class, return_couns = True)
    # print(u)
    # print(c)
    y = u[c == c.max()]
    return y 

  # Function to predict the labels of the test data
  def predict(self,test_data,test_label,k):
    self.test_data = test_data
    self.test_label = test_label
    self.k = k
    prediction=[]
    for x in range(len(self.test_data)):
      pred=self.knn(self.test_data[x],self.k)
      # print(pred)
      self.train_data=np.append(self.train_data,[self.test_data[x]],axis=0)
      self.train_label=np.append(self.train_label,[self.test_label[x]],axis=0)
      prediction.append(pred[0])
    return prediction

# K fold cross validation for the trainining accuracy 
class k_fold_cross_validation:
  def __init__(self):
    pass
  # Function to split data into folds 
  def split_data(self,data,labels,no_of_folds):
    each_fold_size = int(len(data)/no_of_folds)
    j = 0
    data_after_split = []
    labels_after_split = []
    for i in range(no_of_folds):
      fold = []
      fold_labels = []
      fold = data[j:(j + each_fold_size)]
      fold_labels = labels[j:(j + each_fold_size)]
      j += each_fold_size
      data_after_split.append(fold)
      labels_after_split.append(fold_labels)
    return data_after_split,labels_after_split
  
  # Function to evaluate the knn classifier using cross validation
  def evaluate(self,data,labels,no_of_folds,k):
    data_folds,label_folds = self.split_data(data,labels,no_of_folds)
    # print(len(label_folds))
    scores = []
    for i in range(no_of_folds):
      train_data = data_folds.copy()
      train_labels = label_folds.copy()
      test_data = data_folds[i]
      test_labels = label_folds[i]
      del train_data[i]
      del train_labels[i]
      # print(np.vstack((train_data[0],train_data[1])))
      temp_data_array = train_data[0]
      temp_label_array = train_labels[0]
      for j in range(len(train_data)-1):
        temp_data_array = np.vstack((temp_data_array,train_data[j+1]))
        temp_label_array = np.vstack((temp_label_array,train_labels[j+1]))
        # train_label_stack.append(np.vstack((train_labels[j],train_labels[j+1])))
      # final_train_data = train_data_stack
      # print(len(temp_data_array))
      final_train_labels = np.resize(temp_label_array,(1,len(temp_data_array)))[0]
      final_train_data = temp_data_array
      # print(type(final_train_data))
      # print(type(final_train_labels))
      # print(len(final_train_data))
      # print(len(final_train_labels))
      knn = knnclassifier(final_train_data,final_train_labels)
      predictions = knn.predict(test_data,test_labels,k)
      accuracy = ((predictions == test_labels).mean())*100
      scores.append(accuracy)
      # print(scores)
    # print('scores:',scores)
    # print('max accuracy:',max(scores))
    # print('mean accuracy:',sum(scores)/len(scores))
    return sum(scores)/len(scores)



Training knn classifier 

In [ ]:
start = time.time()
kfcv = k_fold_cross_validation()
# k_values = []
# accuracy_values = []
# for k in range(1,20):
knn_training_accuracy = kfcv.evaluate(training_data,training_label,10,6)
  # k_values.append(k)
  # accuracy_values.append(mean_accuracy)
end = time.time()
knn_training_time = time.strftime("%H:%M:%S", time.gmtime(end-start))
print('Training Accuracy:',knn_training_accuracy)
print('Training time:',knn_training_time)


Training Accuracy: 85.55
Training time: 00:16:43


In [ ]:
# knn = knnclassifier(training_data,training_label)
# predictions = knn.predict(validation_data,validation_label,6)
# validation_accuracy = (predictions == validation_label).mean() * 100

# print(validation_accuracy)

knn Testing 

In [ ]:
start = time.time()
knn = knnclassifier(training_data,training_label)
testing_data = testing_data[:2000]
knn_testing_predictions = knn.predict(testing_data,testing_label,6)
testing_predictions = knn_testing_predictions[:2000]
knn_testing_accuracy = (testing_predictions == testing_label).mean() * 100
end = time.time()
print('Testing Accuracy:',knn_testing_accuracy)
knn_running_time = time.strftime("%H:%M:%S", time.gmtime(end-start))
print('Running time:',knn_running_time)

Testing Accuracy: 73.0
Running time: 00:01:11


In [ ]:
training_data = data_train_std_pca
testing_data = data_test_std_pca
training_label = label_train
testing_label = label_test

print(training_data.shape)
print(training_label.shape)
print(testing_data.shape)
print(testing_label.shape)

(30000, 187)
(30000,)
(5000, 187)
(2000,)


Naive Bayes 

In [ ]:
class cross_validation:
  def __init__(self):
    pass
  # Function to split data into folds 
  def split_data(self,data,labels,no_of_folds):
    each_fold_size = int(len(data)/no_of_folds)
    j = 0
    data_after_split = []
    labels_after_split = []
    for i in range(no_of_folds):
      fold = []
      fold_labels = []
      fold = data[j:(j + each_fold_size)]
      fold_labels = labels[j:(j + each_fold_size)]
      j += each_fold_size
      data_after_split.append(fold)
      labels_after_split.append(fold_labels)
    return data_after_split,labels_after_split

  # Function to evaluate and calculate the training accuracy using cross validation
  def evaluate(self,data,labels,no_of_folds):
    data_folds,label_folds = self.split_data(data,labels,no_of_folds)
    # print(len(label_folds))
    scores = []
    for i in range(no_of_folds):
      train_data = data_folds.copy()
      train_labels = label_folds.copy()
      test_data = data_folds[i]
      test_labels = label_folds[i]
      del train_data[i]
      del train_labels[i]
      # print(np.vstack((train_data[0],train_data[1])))
      temp_data_array = train_data[0]
      temp_label_array = train_labels[0]
      for j in range(len(train_data)-1):
        temp_data_array = np.vstack((temp_data_array,train_data[j+1]))
        temp_label_array = np.vstack((temp_label_array,train_labels[j+1]))
        # train_label_stack.append(np.vstack((train_labels[j],train_labels[j+1])))
      # final_train_data = train_data_stack
      # print(len(temp_data_array))
      final_train_labels = np.resize(temp_label_array,(1,len(temp_data_array)))[0]
      final_train_data = temp_data_array
      # print(type(final_train_data))
      # print(type(final_train_labels))
      # print(len(final_train_data))
      # print(len(final_train_labels))
      predictions = naive_bayes(final_train_data,final_train_labels,test_data,test_labels)
      accuracy = ((predictions == test_labels).mean())*100
      scores.append(accuracy)
      # print(scores)
    # print('scores:',scores)
    # print('max accuracy:',max(scores))
    # print('mean accuracy:',sum(scores)/len(scores))
    return sum(scores)/len(scores)

# Function to split data by class values and return a dictionary of class as keys and data as respective values
def separate_class(x_train,y_train) :
  dict1 = {}
  for i in range(len(x_train)):
    
    class_label = y_train[i]
    if class_label not in dict1:
      dict1[class_label] = list()
    dict1[class_label].append(x_train[i])
  return dict1

# Function to calculate prior probabilities and the mean and variance of each feature of each class
def statistics(class_dict):
  classes=list()
  number_in_each_class = list()
  for key,value in class_dict.items():
    classes.append(key)
    number_in_each_class.append(len(value))
  # print(classes)
  # print(number_in_each_class)
  class_wise_numbers = [x for _,x in sorted(zip(classes,number_in_each_class))]
  # print('class wise numbers:',class_wise_numbers)
  classes_sorted = sorted(classes)
  # print('classes:',classes_sorted)
  total = np.sum(np.array(class_wise_numbers))
  # print(total)
  prior_probabilities = np.array(class_wise_numbers)/total
  # print(prior_probabilities)

  #Likelihood 
  class_means = {}
  class_var = {}
  for key,value in class_dict.items():
    means = []
    var = []
    for column in zip(*value):
      means.append(np.mean(column))
      var.append(np.var(column)+1e-3)
    class_means[key] = means
    class_var[key] = var
  class_means_sorted = dict(sorted(class_means.items()))
  class_var_sorted = dict(sorted(class_var.items()))
  stats = {}
  # count = 0 
  for value in classes_sorted:
    # count +=1
    temp_list = []
    for i in range(187):
      temp_tuple = ()
      temp_tuple = (class_means_sorted[value][i],) + (class_var_sorted[value][i],) 
      temp_list.append(temp_tuple)
    stats[value] = temp_list
  # print(count)
  return stats, prior_probabilities

# Function to calculate the gaussian distribution probability
def probability(sample,mean,variance):
    p = 1/(np.sqrt(2*np.pi*variance)) * np.exp((-(sample-mean)**2)/(2*variance))
    return p
# Function to predict the probability of belonging to a class given the features
def class_probability(row, stats,prior_probabilities):
  # total_rows = sum([stats[label][0][2] for label in stats])
  probabilities = {}
  # count = 0 
  for key, value in stats.items():
		# probabilities[key] = stats[key][0][2]/float(total_rows) # prior probability 
    probabilities[key] = prior_probabilities[key]
    for i in range(len(value)):
      mean, var = stats[key][i]
      probabilities[key] *= probability(row[i], mean, var)
    # count +=1
  return probabilities

# Function to Predict the best class for a given row
def predict(stats,row,prior_probabilities):
  probabilities = class_probability(row,stats,prior_probabilities)
  values = list(probabilities.values())
  keys = list(probabilities.keys())
  best_class =  keys[values.index(max(values))]
  return best_class


def naive_bayes(train, y_train,test,y_test):
  all_dictionary = separate_class(train,y_train)
  stats,prior_probabilities = statistics(all_dictionary)
  # print(stats)
  predictions = list()
  for row in test:
    output = predict(stats, row,prior_probabilities)
    predictions.append(output)
  return predictions
 


Training

In [ ]:
start = time.time()
cv = cross_validation()
nb_training_accuracy = cv.evaluate(training_data,training_label,10)
end = time.time()
nb_training_time = time.strftime("%H:%M:%S", time.gmtime(end-start))
print('Training Accuracy:',nb_training_accuracy)
print('Training time:',nb_training_time)

Training Accuracy: 74.40333333333332
Training time: 00:05:00


Testing 

In [ ]:
start = time.time()
training_data = training_data[:2000]
nb_testing_predictions = naive_bayes(training_data,training_label,testing_data,testing_label)
nb_testing_accuracy = (nb_testing_predictions == testing_label).mean() * 100
end = time.time()
print('Accuracy:',nb_testing_accuracy)
print('Running time:',end-start)

Data Load 

In [ ]:
training_data = data_train_std_pca
testing_data = data_test_std_pca
training_label = label_train
testing_label = label_test

print(training_data.shape)
print(training_label.shape)
print(testing_data.shape)
print(testing_label.shape)

(30000, 187)
(30000,)
(5000, 187)
(2000,)


Logistic Regression

In [ ]:
# Function to add bias to the input
def add_bias(x):
    return np.hstack((np.ones((x.shape[0],1)), x))

inputs = add_bias(training_data)
labels = training_label[:,np.newaxis]
# print(inputs.shape)
# print(labels.shape)

# Initializing weights
weights = np.zeros((inputs.shape[1], 1))
# weights = np.random.random((inputs.shape[1],1))
# print(weights.shape)

# Function to calculate the probability of the model
def probability_of_linear_model(inputs, weights):
    z = np.dot(inputs, weights)
    exp_z = np.exp(z)
    p = exp_z/(1 + exp_z)
    return p

# Function to calculate the log likelihood loss which is minimized to obtain optimal weights
def log_likelihood_loss(weights,inputs,labels):
  n = inputs.shape[0]
  # p = probability_of_linear_model(inputs,weights)
  z = np.dot(inputs, weights)
  exp_z = np.exp(z)
  # print(exp_z)
  # loss = -(1/n) * np.sum(labels*np.log(p) + (1-labels)*np.log(1-p))
  loss =  (1/n)*np.sum(np.log(1.0+exp_z)+(labels*z))
  return loss
# loss = log_likelihood_loss(weights,inputs,labels)
# print(loss)

# Function to find the gradient
def gradient(weights,inputs,labels):
  n = inputs.shape[0]
  p = probability_of_linear_model(inputs,weights)
  gradient = (1/n) * np.dot(inputs.T, p - labels)
  return gradient 
# grad = gradient(weights,inputs,labels)
# print(grad)

# Function to the fit the model: finding optimized weights by minimizing the loss 
def fit(inputs, labels, weights):
    # optimal_weigths = fmin_tnc(func=log_likelihood_loss,x0=weights,fprime=gradient, args=(inputs, labels.flatten()))
    # optimal_weigths = minimize(fun=log_likelihood_loss,x0=weights,args=(inputs, labels.flatten()),method='BFGS',jac=gradient)
    # optimal_weigths = fmin(func=log_likelihood_loss,x0=weights,args=(inputs, labels.flatten()),xtol=1e-6,ftol=1e-6)
    optimal_weigths = fmin_cobyla(func=log_likelihood_loss,x0=weights,cons=gradient,args=(inputs, labels.flatten()))
    # optimal_weigths = fmin_tnc(func=log_likelihood_loss, x0=weights,approx_grad=True, args=(inputs, labels.flatten())) -> Taking time for gradient calculation
    return optimal_weigths
# def fit(inputs, labels, weights,niter,eta):
#     for i in niter:
#       loss = log_likelihood_loss(weights,inputs,labels)
#       gradient = gradient(weights,inputs,labels)
#       weights = weights - gradient * eta
#     return optimal_weigths
# weights_fit = fit(inputs,labels,weights)
# print(weights_fit)

def make_columns_for_each_class(labels,class_value):
  labels_new = []
  for i in range(0, len(labels)):
    if labels[i] == class_value:
      labels_new.append(1)
    else:
      labels_new.append(0)
  return labels_new

def find_weights(inputs, labels, weights):
    unique_labels = list(set(labels.flatten()))
    weights_list = []
    for i in unique_labels:
        weight_new = fit(inputs, labels, weights)
        weights_list.append(weight_new)
    return weights_list

def predict(weights_list, x, y):
    y_classes = list(set(y.flatten()))
    y_hat = [0]*len(y)
    for i in range(0, len(y_classes)):
        y_new = make_columns_for_each_class(y, y_classes[i])
        z = np.dot(x, weights_list[i])
        exp_z = np.exp(z)
        p = exp_z/(1 + exp_z)
        for k in range(0, len(y)):
            if y_new[k] == 1 and p[k] >= 0.5:
                y_hat[k] = y_classes[i]
    return y_hat




Training 

In [ ]:
start = time.time()
weights_list = find_weights(inputs, labels, weights)
# print(weights_list)
y_hat = predict(weights_list, inputs, labels)
predicted = np.array(y_hat)[:,np.newaxis]
# print(predicted.shape)
lr_training_accuracy = (predicted == labels).mean() * 100 
end = time.time()
lr_training_time = time.strftime("%H:%M:%S", time.gmtime(end-start))
print('Training Accuracy:',lr_training_accuracy)
print('Training time:',lr_training_time)

Training Accuracy: 88.26333333333334
Training time: 00:07:39


Testing

In [ ]:
# start = time.time()
# testing_data = testing_data[:2000]
# test_inputs = add_bias(testing_data)
# test_labels = testing_label[:,np.newaxis]
# test_y_hat = predict(weights_list, test_inputs, test_labels)
# test_predicted = np.array(test_y_hat)[:,np.newaxis]
# lr_test_accuracy = (test_predicted == test_labels).mean() * 100 
# end = time.time()
# print('Accuracy:',lr_test_accuracy)
# print('Running time:',end-start,'seconds')



Accuracy: 77.2
Running time: 0.08029031753540039 seconds


Prediction and Testing

In [ ]:
start = time.time()
test_inputs = add_bias(testing_data)
test_labels = testing_label[:,np.newaxis]
test_y_hat = predict(weights_list, test_inputs, test_labels)
test_predicted = np.array(test_y_hat)[:,np.newaxis]
lr_test_accuracy = (test_predicted == test_labels).mean() * 100 
end = time.time()
print('Accuracy:',lr_test_accuracy)
print('Running time:',end-start,'seconds')


Exporting data to file

In [ ]:
output = test_predicted
with h5py.File('predicted_labels.h5','w') as H:
  H.create_dataset('Output',data=output)

Comparison between classifier algorithms

In [ ]:
from astropy.table import  Table, Column

data_rows = [('KNN', knn_training_accuracy, knn_training_time),
             ('Naive Bayes', round(nb_training_accuracy,2), nb_training_time),
             ('Logistic Regression', round(lr_training_accuracy,2), lr_training_time)]
t = Table(rows=data_rows, names=('Algorithm', 'Training Accuracy', 'Training Time'))
print(t)

     Algorithm      Training Accuracy Training Time
------------------- ----------------- -------------
                KNN             85.55      00:16:43
        Naive Bayes              74.4      00:05:00
Logistic Regression             88.26      00:07:39


Hardware and Software Specifications




Reference: [Specifications](https://www.thepythoncode.com/article/get-hardware-system-information-python)

In [ ]:
import platform
import psutil
# System Info
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")
# CPU 
print('Coding enviroment: Google Colab')
print(f"Total CPU Usage: {psutil.cpu_percent()}%")
# Memory 
svmem = psutil.virtual_memory()
print(f"Total: {svmem.total} bytes")
print(f"Available: {svmem.available} bytes")
print(f"Used: {svmem.used} bytes")


System: Linux
Node Name: 69f58f51a834
Release: 4.19.112+
Version: #1 SMP Thu Jul 23 08:00:38 PDT 2020
Machine: x86_64
Processor: x86_64
Coding enviroment: Google Colab
Total CPU Usage: 1.3%
Total: 13653557248 bytes
Available: 12526338048 bytes
Used: 8414552064 bytes
